# Acceso a ECCOv4 por medio de EarthData in the Cloud de la NASA

Este tutorial demuestra acceso al producto [ECCOv4](https://ecco-group.org/) de un modelo numerico global. Para mayor informacion del producto de ECCO, puede ir [aqui](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO).

**Requisitos**
1. Tener una cuenta de Earth Data Login por medio de la NASA.
2. Tener un Token valido.

Tambien se puede utilizar el metodo de Nombre de Usuario / Contrasena descrito en el tutorial de [Autenticacion](Authentication)


**Objectivos**
 
Utilizar [pydap](https://pydap.github.io/pydap/) para demostrar

- Acceso to archivos cientificos de la  NASA por medio del uso de `tokens` y [EarthData](https://www.earthdata.nasa.gov/) como metodo de autenticacion. 


Algunas variables de interes son:

- [Native grid](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4)
- [Temperature and Salinity](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_TEMP_SALINITY_LLC0090GRID_MONTHLY_V4R4)



`Autor`: Miguel Jimenez-Urias, '24

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydap.net import create_session
from pydap.client import open_url
import xarray as xr
import pydap


In [ ]:
print("xarray version: ", xr.__version__)
print("pydap version: ", pydap.__version__)

### Acceso a EARTHDATA

El listado de las variables de este producto pueden encontrarse [aqui](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO). En este caso, accederemos a las variables en su malla original, el LL90.

In [ ]:
Grid_url = 'https://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/ECCO%20Geometry%20Parameters%20for%20the%20Lat-Lon-Cap%2090%20(llc90)%20Native%20Model%20Grid%20(Version%204%20Release%204)/granules/GRID_GEOMETRY_ECCO_V4r4_native_llc0090'

### Autenticacion via `.netrc`

Las credenciales son recuperadas automaticamente por `pydap`.


In [ ]:
my_session = create_session()

### Alternativa: El use the tokens
```python
session_extra = {"token": "YourToken"}

# initialize a requests.session object with the token headers. All handled by pydap.
my_session = create_session(session_kwargs=session_extra)
```

### Accesso a los metadatos solamente, por medio de pydap

`pydap` aprovecha el protocolo de OPeNDAP, el cual permite la separacion de los metadatos de los valores numericos. Esto permite una inspeccion remota de los datasets.

In [ ]:
ds_grid = open_url(Grid_url, session=my_session, protocol="dap4")

In [ ]:
ds_grid.tree()

```{note}
PyDAP accesa solo a los metadatos de archivo en el servidor de OPeNDAP, y ningun arreglo numerico ha sido descargardo hasta este punto!
```

**Para descargar los arreglos numericos hay que indexar la variable**

Esto es, de la siguiente manera:

```python
# this fetches remote data into a pydap object container
pydap_Array = dataset[<VarName>][:]
```
donde `<VarName>` es el nombre de una de las variables. El producto sera una representation "en memoria" de tipo `pydap.model.BaseType`, el cual permite el acceso a los arreglos de `numpy`. 

Para extraer los valores de la variable remota, hay que ejecutar el siguiente comando

```python
# The `.data` command allows direct access to the Numpy array (e.g. for manipulation)
pydap_Array.data
```



In [ ]:
# lets download some data
Depth = ds_grid['Depth'][:]
print(type(Depth))

In [ ]:
Depth.attributes

In [ ]:
Depth.shape, Depth.dims

**Visualizando el fondo oceanico `Depth` en la malla original del modelo**

En este caso, el producto `ECCO` esta definido en una malla con topologia de un `Cubo Esferico`. De esta manera, la malla horizontal contiene una dimension extra llamada: `tile` o `face`. A continuacion visualizamos la variable en su topologia original

In [ ]:
Variable = [Depth[i].data for i in range(13)]
clevels =  np.linspace(0, 6000, 100)
cMap = 'Greys_r'

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES = [
    axes[4, 0], axes[3, 0], axes[2, 0], axes[4, 1], axes[3, 1], axes[2, 1],
    axes[1, 1], 
    axes[1, 2], axes[1, 3], axes[1, 4], axes[0, 2], axes[0, 3], axes[0, 4],
]
for i in range(len(AXES)):
    AXES[i].contourf(Variable[i], clevels, cmap=cMap)

for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

**Fig. 1.** La variable `Depth` visualizada en una malla horitonzal. `Tile`s con valores `0-5` tienen un ordenamiento de valores (indiciales) `C-ordering`, mientras que cualquier arreglo numerico `tile`s `7-13` siguen el `F-ordering`. Cualquier arreglo numerico en el `arctic cap`, tiene un comportamiento de una coordenada polar.

**Visualizacion con topologia corregida**

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES_NR = [
    axes[3, 0], axes[2, 0], axes[1, 0], axes[3, 1], axes[2, 1], axes[1, 1],
]
AXES_CAP = [axes[0, 0]]
AXES_R = [
    axes[1, 2], axes[2, 2], axes[3, 2], axes[1, 3], axes[2, 3], axes[3, 3],
]
for i in range(len(AXES_NR)):
    AXES_NR[i].contourf(Variable[i], clevels, cmap=cMap)

for i in range(len(AXES_CAP)):
    AXES_CAP[i].contourf(Variable[6].transpose()[:, ::-1], clevels, cmap=cMap)

for i in range(len(AXES_R)):
    AXES_R[i].contourf(Variable[7+i].transpose()[::-1, :], clevels, cmap=cMap)

for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

**Fig. 2.** Visualizacion de la variable `Depth` en una malla horizontal que approxima una malla horizontal uniforme con ejes `lat-lon`. Sin embargo, cualquier arreglo numerico en el `arctic cap`, continua en una malla que approxima una malla en coordenadas polares

### Utilizando `xarray`

`pydap` se puede llamar dentro de `xarray` para acceder a los archivos expuestos mediante el servidor de OPeNDAP. En particular, `xarray` permite aggregar multiples archivos remotos de opendap. A continuacion demostramos un pequeno ejemplo.




**Un URL individual --> un archivo remoto**

Below we access remote Temperature and Salinity ECCO data with `xarray` via (internally) `pydap`.


In [ ]:
baseURL = 'https://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/'
Temp_Salt = "ECCO%20Ocean%20Temperature%20and%20Salinity%20-%20Monthly%20Mean%20llc90%20Grid%20(Version%204%20Release%204)/granules/OCEAN_TEMPERATURE_SALINITY_mon_mean_"
year = '2017-'
month = '01'
end_ = '_ECCO_V4r4_native_llc0090'
Temp_2017 = baseURL + Temp_Salt +year  + month + end_

# Convertimos el URL en un DAP4 opendap url
Temp_2017 = Temp_2017.replace("https", "dap4")
Temp_2017

In [ ]:
pyds = open_url(Temp_2017, session=my_session)
pyds.tree()

Para evitar descargas variables que no sean necesariamente de interest, se le puede instruir al Servidor Hyrax que variables requiere uno.

Para esto utilizaremos las `Expresiones de Restriccion` (CE, por sus siglas en ingles). Este metodo nos ayudara a construir datasets mucho mas simples, 
y evitar descargar N veces, informacion que no es requerida.

A continuacion demostramos el caso de solo requerir la variable `THETA` y sus dimensiones.


In [ ]:
dims = pyds['/THETA'].dims
Vars = ['/THETA'] + dims

# Below construct Contraint Expression
CE = "?dap4.ce="+(";").join(Vars)
print("Constraint Expression: ", CE)

In [ ]:
Temp_2017 = baseURL + Temp_Salt +year  + month + end_ + CE
Temp_2017

### Importante

Ahora aplicamos la CE a todos los possibles URLs the nuestro dataset.

In [ ]:
Temp_2017 = [baseURL.replace("https", "dap4") + Temp_Salt + year + f'{i:02}' + end_ + CE for i in range(1, 13)]
Temp_2017[:3]

###

Debajo inicializaremos una session que persista, cada vez que descargue datos de servidores de OPeNDAP. En este caso utilizaremost la libraria
`requests_cache`. El comportamiento es casi identico excepto la siguiente celda:


In [ ]:
cache_session = create_session(use_cache=True) # caching the session
cache_session.cache.clear() # clear all cache to demonstrate the behavior

```{warning}
A partir de la version de `pydap >= 3.5.5`, existe el nuevo metodo experimental `consolidated_metadata` que permite a pydap descargar todos los metadatos, y reusarlos. Este metodo continua en desarrollo y sus propiedades pueden cambiar en las siguientes versiones.
```


In [ ]:
from pydap.client import consolidate_metadata

In [ ]:
%%time
consolidate_metadata(Temp_2017, session=cache_session, concat_dim='time')

In [ ]:
%%time
theta_salt_ds = xr.open_mfdataset(
    Temp_2017, 
    engine='pydap',
    session=cache_session, 
    parallel=True, 
    combine='nested', 
    concat_dim='time',
)

In [ ]:
theta_salt_ds

### Finalmente, visualizamos `THETA`

In [ ]:
Variable = [theta_salt_ds['THETA'][0, 0, i, :, :] for i in range(13)]
clevels = np.linspace(-5, 30, 100)
cMap='RdBu_r'

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES_NR = [
    axes[3, 0], axes[2, 0], axes[1, 0], axes[3, 1], axes[2, 1], axes[1, 1],
]
AXES_CAP = [axes[0, 0]]
AXES_R = [
    axes[1, 2], axes[2, 2], axes[3, 2], axes[1, 3], axes[2, 3], axes[3, 3],
]
for i in range(len(AXES_NR)):
    AXES_NR[i].contourf(Variable[i], clevels, cmap=cMap)

for i in range(len(AXES_CAP)):
    AXES_CAP[i].contourf(Variable[6].transpose()[:, ::-1], clevels, cmap=cMap)

for i in range(len(AXES_R)):
    AXES_R[i].contourf(Variable[7+i].transpose()[::-1, :], clevels, cmap=cMap)

for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

**Fig. 3.** Visualizacion de la variable `Surface temperature`, similar al metodo utilizado en la Figura 2.